In [68]:
from bicom_packet import *
from binascii import *
import math
from utils import *
import serial 
from osrd_ptz import *
import time
import angles
from rcp_ptz import *
import numpy as np
from numpy import array as a
import scipy as sp
import scipy.interpolate as spi

In [2]:
conn = {
    'url': 'http://10.0.2.5/rcp.xml',
}
limits = dotdict({
    'az_off': 0,
    'az_flip': False,
    'el_off': 0,
    'z_off': 0,
})

In [ ]:
ser = serial.Serial('COM23', 57600)
ptz = OSRDPTZ(ser, 0, limits)

In [3]:
ptz = RCPPTZ(conn, limits)

In [4]:
for i in range(0, 361, 60):
    print(f'moving to {i}')
    ptz.move(i, 0, None)
    time.sleep(1)

moving to 0
moving to 60
moving to 120
moving to 180
moving to 240
moving to 300
moving to 360


In [ ]:
ptz.move(180,60,None)

In [ ]:
BiCom(bytes(ptz.move(0,0,None)))

In [ ]:
bytes(pkt).hex()

In [94]:
zmap = {}

In [95]:
elz = a([(60, 0), (50, 20), (40, 40), (30, 60), (20, 80), (10, 100), (0, 120)], dtype=[('x', float), ('y', float)])
elz.sort(axis=0, order='x')
x, y = elz['x'], elz['y']
f = spi.interp1d(x, y, bounds_error=False, fill_value=(y[0], None), assume_sorted=True)
for num in [1, 2, 3, 4, 5, 6]:
    zmap[f'Bosch{num}'] = f

In [96]:
elz = a([(60, 0), (50, 10), (40, 20), (30, 30), (20, 40), (10, 50), (0, 40)], dtype=[('x', float), ('y', float)])
elz.sort(axis=0, order='x')
x, y = elz['x'], elz['y']
f = spi.interp1d(x, y, bounds_error=False, fill_value=(y[0], None), assume_sorted=True)
for num in [7, 8, 9, 14, 15]:
    zmap[f'Bosch{num}'] = f

In [103]:
f = np.vectorize(lambda x: np.NaN)
for num in [10, 11, 12, 16, 17, 18]:
    zmap[f'Bosch{num}'] = f

In [104]:
zmap

{'Bosch1': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438b38>,
 'Bosch2': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438b38>,
 'Bosch3': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438b38>,
 'Bosch4': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438b38>,
 'Bosch5': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438b38>,
 'Bosch6': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438b38>,
 'Bosch7': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438a98>,
 'Bosch8': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438a98>,
 'Bosch9': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438a98>,
 'Bosch14': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438a98>,
 'Bosch15': <scipy.interpolate.interpolate.interp1d at 0x2b3ac438a98>,
 'Bosch10': <numpy.lib.function_base.vectorize at 0x2b3b10b43c8>,
 'Bosch11': <numpy.lib.function_base.vectorize at 0x2b3b10b43c8>,
 'Bosch12': <numpy.lib.function_base.vectorize at 0x2b3b10b43c8>,
 'Bosch16': <numpy.lib.functio

In [106]:
zmap['Bosch16'](2)

array(nan)

In [32]:
elz = a([[60, 0], [50, 10], [40, 20], [30, 30], [20, 40], [10, 50], [0, 40]])
elz.sort(axis=0)
f = lambda el: np.interp(el, elz[:,0], elz[:,1], right=None)
for num in [1, 2, 3, 4, 5, 6]:
    zmap[f'Bosch{num}'] = f

27.4

In [88]:
f(-1)

array(120.)

In [111]:
round(float(zmap['Bosch16'](2)))

ValueError: cannot convert float NaN to integer